In [1]:
import pymongo
import pandas as pd
import numpy as np
from glob import glob
import subprocess
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime, timedelta
from scipy.optimize import brentq
from scipy.stats import norm
from math import exp, log, sqrt

# Find Expiry path all index expirys

In [36]:
def find_expiry_path(today_date_str):
    month = datetime.today().month
    year = datetime.today().year
    data_path = f'Y:\\daily_contract_file\\{today_date_str}\\*'
    
    # Read file paths into a DataFrame
    file_paths = pd.DataFrame(glob(data_path), columns=['path'])
    
    # Extract split_data, index_name, and expiry using a single apply
    file_paths['split_data'] = file_paths['path'].apply(lambda x: x.split('\\')[-1])
    file_paths[['index_name', 'expiry']] = file_paths['split_data'].str.extract(r'([^_]+)_(\d{8})')
    
    # Remove rows where expiry is 'index'
    file_paths = file_paths[file_paths['expiry'] != 'index']
    
    # Convert 'expiry' to datetime and sort the DataFrame
    file_paths['expiry'] = pd.to_datetime(file_paths['expiry'], format='%d%m%Y')
    file_paths = file_paths.sort_values(by='expiry')
    
    # Create separate DataFrames for each index_name
    index_names = ['BANKNIFTY', 'NIFTY', 'FINNIFTY', 'MIDCPNIFTY']
    separated_data = {index: file_paths[file_paths['index_name'] == index].reset_index(drop=True)
                      for index in index_names}
    
    # Accessing the DataFrames
    bank_nifty = separated_data['BANKNIFTY']
    nifty = separated_data['NIFTY']
    fin_nifty = separated_data['FINNIFTY']
    mid_nifty = separated_data['MIDCPNIFTY']
    index_list = [bank_nifty ,nifty , fin_nifty , mid_nifty ]
    weekly_list = {}
    next_weekly_list = {}
    monthly_list = {}
    
    for i in index_list :
        try:
            name = i['index_name'].iloc[0]
            weekly  = i['path'][0]
            next_weekly = i['path'][1]
            monthly = i[(i['expiry'].dt.month == month) & (i['expiry'].dt.year == year)].reset_index(drop = 'First')['path'].iloc[-1]
        except Exception as e:
            monthly = i[(i['expiry'].dt.month == (month + 1)) & (i['expiry'].dt.year == year)].reset_index(drop = 'First')['path'].iloc[-1]
        
    exp_name = ['weekly' , 'next_weekly' , 'monthly']
    exp_path_list = [weekly_list ,next_weekly_list , monthly_list]
    return exp_path_list

# Find Token Numbers with token details all indexs expirys

In [3]:
def find_token_data(exp_path_list):
    exp_name = ['weekly' , 'next_weekly' , 'monthly']
    for i, path in enumerate(exp_path_list):
        print(path)
        print(exp_name[i])

        # Find Weekly token details (Bank Nifty, Nifty , Fin-Nifty , Mid-Nifty)
        if exp_name[i] == 'weekly':
            
            for key,value in path.items():
                
                if key == 'BANKNIFTY':
                    bank_nifty_token_dic_weekly = pd.read_csv(value , dtype={'Expiry date': object})
    
                elif key == 'NIFTY':
                    nifty_token_dic_weekly = pd.read_csv(value , dtype={'Expiry date': object})
                  
                elif key == 'FINNIFTY':
                    fin_nifty_token_dic_weekly = pd.read_csv(value , dtype={'Expiry date': object})
                   
                else:
                    mid_nifty_token_dic_weekly = pd.read_csv(value , dtype={'Expiry date': object})
                    
         # Find Next-Weekly token details (Bank Nifty, Nifty , Fin-Nifty , Mid-Nifty)           
        elif exp_name[i] == 'next_weekly' :
            
            for key,value in path.items():
                
                if key == 'BANKNIFTY':
                    bank_nifty_token_dic_next_weekly = pd.read_csv(value , dtype={'Expiry date': object})
                    
                elif key == 'NIFTY':
                    nifty_token_dic_next_weekly = pd.read_csv(value , dtype={'Expiry date': object}) 
                    
                elif key == 'FINNIFTY':
                    fin_nifty_token_dic_next_weekly = pd.read_csv(value , dtype={'Expiry date': object})
                   
                else:
                    mid_nifty_token_dic_next_weekly = pd.read_csv(value , dtype={'Expiry date': object})
                    
         # Find Monthly token details (Bank Nifty, Nifty , Fin-Nifty , Mid-Nifty)
        elif exp_name[i] == 'monthly':
            for key,value in path.items():
                
                if key == 'BANKNIFTY':
                    bank_nifty_token_dic_monthly = pd.read_csv(value , dtype={'Expiry date': object})
                    
                elif key == 'NIFTY':
                    nifty_token_dic_monthly = pd.read_csv(value , dtype={'Expiry date': object})
                   
                    
                elif key == 'FINNIFTY':
                    fin_nifty_token_dic_monthly = pd.read_csv(value , dtype={'Expiry date': object})
                   
                else:
                    mid_nifty_token_dic_monthly = pd.read_csv(value , dtype={'Expiry date': object})

    token_dics = {
                    'weekly': {'BANKNIFTY': bank_nifty_token_dic_weekly, 'NIFTY': nifty_token_dic_weekly, 
                               'FINNIFTY': fin_nifty_token_dic_weekly, 'MIDNIFTY': mid_nifty_token_dic_weekly},
                    'next_weekly': {'BANKNIFTY': bank_nifty_token_dic_next_weekly, 'NIFTY': nifty_token_dic_next_weekly, 
                                    'FINNIFTY': fin_nifty_token_dic_next_weekly, 'MIDNIFTY': mid_nifty_token_dic_next_weekly},
                    'monthly': {'BANKNIFTY': bank_nifty_token_dic_monthly, 'NIFTY': nifty_token_dic_monthly, 
                                
                                'FINNIFTY': fin_nifty_token_dic_monthly, 'MIDNIFTY': mid_nifty_token_dic_monthly}
                }
    return token_dics

# Find Future Token Number comman funcation of all indexs

In [4]:
def fut_price_token(today_date_str , index_name):
    fut_file = f'Y:\\daily_contract_file\\{today_date_str}\\future_index.csv'
    fut_data = pd.read_csv(fut_file)
    fut_data['month_exp'] = pd.to_datetime(fut_data['Expiry date'], format='%d%m%Y').dt.month
    fut_data = fut_data[fut_data['Symbol'] == index_name]
    token_number_fut = fut_data.sort_values(by = 'month_exp',ascending = True).reset_index(drop = 'First').iloc[0]['Token']
    return token_number_fut

# segregate option data of all indexes and all expiry-wise 

In [5]:
def process_data(file_path, token_dics, bank_fut_token, nifty_fut_token, fin_nifty_fut_token, mid_nifty_fut_token, chunk_size=100000):
    # Initialize your lists
    data_dict = {
        'weekly_bank_nifty_data': [],
        'next_weekly_bank_nifty_data': [],
        'monthly_bank_nifty_data': [],
        'bank_fut_price': [],
        'weekly_nifty_data': [],
        'next_weekly_nifty_data': [],
        'monthly_nifty_data': [],
        'nifty_fut_price': [],
        'weekly_fin_nifty_data': [],
        'next_weekly_fin_nifty_data': [],
        'monthly_fin_nifty_data': [],
        'fin_nifty_fut_price': [],
        'weekly_mid_nifty_data': [],
        'next_weekly_mid_nifty_data': [],
        'monthly_mid_nifty_data': [],
        'mid_nifty_fut_price': []
    }

    # Function to clean data
    def clean_data(data):
        new_name = {
            0: 'Token',
            2: 'time_entry',
            3: 'volume',
            4: 'open',
            5: 'high',
            6: 'low',
            7: 'prev-close',
            8: 'ltp',
            9: 'atp',
            13: 'bid_price_1',
            15: 'ask_price_1'
        }
        data.rename(new_name, axis=1, inplace=True)
        data['Token'] = data['Token'].str[23:-1]
        data['time_entry'] = data['time_entry'].str[16:-1]
        data['volume'] = data['volume'].str[11:]
        data['open'] = data['open'].str[9:]
        data['high'] = data['high'].str[9:]
        data['low'] = data['low'].str[8:]
        data['prev-close'] = data['prev-close'].str[15:]
        data['ltp'] = data['ltp'].str[8:]
        data['atp'] = data['atp'].str[8:]
        data['bid_price_1'] = data['bid_price_1'].str[16:]
        data['ask_price_1'] = data['ask_price_1'].str[16:]
        return data

    # Function to process chunks and filter data
    def process_and_filter_data(data, tokens, category, result_list):
        filter_token = [str(i) for i in tokens['Token']]
        filtered_chunk = data[data['Token'].isin(filter_token)]
        result_list.append(filtered_chunk)

    # Read and process data in chunks
    for chunk in pd.read_csv(file_path, header=None, chunksize=chunk_size, encoding='utf-8', usecols=[0, 2, 3, 4, 5, 6, 7, 8, 9, 13, 15]):
        data = clean_data(chunk)

        # Filter future prices
        bank_fut_data = data[data['Token'] == str(bank_fut_token)]
        data_dict['bank_fut_price'].append(bank_fut_data)

        nifty_fut_data = data[data['Token'] == str(nifty_fut_token)]
        data_dict['nifty_fut_price'].append(nifty_fut_data)

        fin_nifty_fut_data = data[data['Token'] == str(fin_nifty_fut_token)]
        data_dict['fin_nifty_fut_price'].append(fin_nifty_fut_data)

        mid_nifty_fut_data = data[data['Token'] == str(mid_nifty_fut_token)]
        data_dict['mid_nifty_fut_price'].append(mid_nifty_fut_data)

        # Process and filter data for each expiration type and index
        for expiration, index_tokens in token_dics.items():
            for index, tokens in index_tokens.items():
                if expiration == 'weekly':
                    if index == 'BANKNIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['weekly_bank_nifty_data'])
                    elif index == 'NIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['weekly_nifty_data'])
                    elif index == 'FINNIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['weekly_fin_nifty_data'])
                    else:  # MIDNIFTY
                        process_and_filter_data(data, tokens, 'Token', data_dict['weekly_mid_nifty_data'])
                elif expiration == 'next_weekly':
                    if index == 'BANKNIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['next_weekly_bank_nifty_data'])
                    elif index == 'NIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['next_weekly_nifty_data'])
                    elif index == 'FINNIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['next_weekly_fin_nifty_data'])
                    else:  # MIDNIFTY
                        process_and_filter_data(data, tokens, 'Token', data_dict['next_weekly_mid_nifty_data'])
                else:  # monthly
                    if index == 'BANKNIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['monthly_bank_nifty_data'])
                    elif index == 'NIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['monthly_nifty_data'])
                    elif index == 'FINNIFTY':
                        process_and_filter_data(data, tokens, 'Token', data_dict['monthly_fin_nifty_data'])
                    else:  # MIDNIFTY
                        process_and_filter_data(data, tokens, 'Token', data_dict['monthly_mid_nifty_data'])

    return data_dict


# Merge tokens deatils and future price in option data 

In [6]:
def token_deatils_merge(data , token_data, fut_data):
    # Taken only required columns 
    token_data = token_data[['Token' , 'strike_price' , 'option_type', 'Expiry date']]

    # change token data type int to str
    token_data['Token'] = token_data['Token'].astype(str)

    # convert expiry date object to datetime type
    token_data['Expiry date'] = pd.to_datetime(token_data['Expiry date'] , format = "%d%m%Y")

    # Merge data
    df = pd.merge(data,token_data , on = 'Token' , how="inner")
    fut_data = fut_data[['time_entry' , 'ltp']]
    fut_data.rename({'ltp': 'future_price'}, axis=1, inplace=True)

    
    fut_data['time_entry'] = pd.to_datetime(fut_data['time_entry'])
    df['time_entry'] = pd.to_datetime(df['time_entry'])
    df = df.sort_values(by='time_entry')
    # Merge Future price in options data
    final_data = pd.merge_asof(df , fut_data , on = 'time_entry' , direction="nearest")
    return final_data

# Merge Spot price only indexs

In [7]:
def spot_price_merge(data , index_name , today_date_str):
    # Read index data , this is a file path format
    index_file = f'Y:\\{today_date_str}\\index_data.txt'
    df= pd.read_csv(index_file , header = None,encoding='utf-8' , usecols=[0,2,3])
    df.rename(columns = {0: 'index_name' , 2 : 'time_entry' , 3 : 'spot_price'},inplace = True)
    
    df['index_name'] = df['index_name'].str[23:-1]
    df['time_entry'] = df['time_entry'].str[16:-1]
    df['spot_price'] = df['spot_price'].str[8:-2]
    
    index_data = df[df['index_name'] == index_name].reset_index(drop ='First')
    index_data.drop(['index_name'] , axis = 1 ,inplace = True)
    index_data['time_entry'] = pd.to_datetime(index_data['time_entry'])
    main_data = pd.merge_asof(data , index_data , on='time_entry' , direction="nearest")
    main_data['Expiry date'] = pd.to_datetime(main_data['Expiry date'] , format = "%d%m%Y")
    exp = main_data['Expiry date'].iloc[0].date()
    entry_date = main_data['time_entry'].iloc[0].date()
    
    # Find day to exp (expiry date coulmns minus current date)
    main_data['day_to_exp'] = (exp - entry_date).days
    return main_data

# Calculate IV of call with spot price 

In [8]:
def black_scholes_call(S, X, r, sigma, t):
    d1 = (log(S / X) + (r + (sigma ** 2) / 2) * t) / (sigma * sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    return S * norm.cdf(d1) - X * exp(-r * t) * norm.cdf(d2)
def implied_volatility_spot_call(row):
    # Extract necessary data from the row
    r = 0.01
    # Calculate S and X (spot price and strike price)
    S = row['spot_price']
    X = row['strike_price']
    C = row['ltp']
    t = row['day_to_exp'] / 365
    # If current date is same as expiry date, time to expiry is 0
    if t == 0:
        t = 0.001  # Avoid division by zero
    else:
        t = t
   
    # Define the objective function for implied volatility
    def f(sigma):
        return black_scholes_call(S, X, r, sigma, t) - C

    try:
        # Compute implied volatility
        implied_volatility = brentq(f, 0.001, 2.0)
        return implied_volatility
    except ValueError:
        return None

# Calculate call IV with Future price 

In [9]:
def implied_volatility_future_call(row):
    # Extract necessary data from the row
    r = 0.01
    # Calculate S and X (spot price and strike price)
    S = row['future_price']
    X = row['strike_price']
    C = row['ltp']
    t = row['day_to_exp'] / 365
    # If current date is same as expiry date, time to expiry is 0
    if t == 0:
        t = 0.001  # Avoid division by zero
    else:
        t = t
    # Define the objective function for implied volatility
    def f(sigma):
        return black_scholes_call(S, X, r, sigma, t) - C

    try:
        # Compute implied volatility
        implied_volatility = brentq(f, 0.001, 2.0)
        return implied_volatility
    except ValueError:
        return None

# Calculate put IV with spot price

In [10]:
def black_scholes_put(S, X, r, sigma, t):
    d1 = (log(S / X) + (r + (sigma ** 2) / 2) * t) / (sigma * sqrt(t))
    d2 = d1 - sigma * sqrt(t)
    return X * exp(-r * t) * norm.cdf(-d2) - S * norm.cdf(-d1)
def implied_volatility_spot_put(row):
    # Extract necessary data from the row
    r = 0.01
    S = row['spot_price']
    X = row['strike_price']
    P = row['ltp']
    t = row['day_to_exp'] / 365
    # If current date is same as expiry date, time to expiry is 0
    if t == 0:
        t = 0.001  # Avoid division by zero
    else:
        t = t
    # Define the objective function for implied volatility
    def f(sigma):
        return black_scholes_put(S, X, r, sigma, t) - P

    try:
        # Compute implied volatility
        implied_volatility = brentq(f, 0.001, 2.0)
        return implied_volatility
    except ValueError:
        return None


# Calculate put IV with Future price

In [11]:
def implied_volatility_future_put(row):
    # Extract necessary data from the row
    r = 0.01
    S = row['future_price']
    X = row['strike_price']
    P = row['ltp']
    t = row['day_to_exp'] / 365
    # If current date is same as expiry date, time to expiry is 0
    if t == 0:
        t = 0.001  # Avoid division by zero
    else:
        t = t
   
    # Define the objective function for implied volatility
    def f(sigma):
        return black_scholes_put(S, X, r, sigma, t) - P

    try:
        # Compute implied volatility
        implied_volatility = brentq(f, 0.001, 2.0)
        return implied_volatility
    except ValueError:
        return None

# Seprate call data and put data and calculate iv and save data in main data 

In [12]:
def calculate_iv(data):
    # change data type str to float
    data[['ltp', 'spot_price', 'future_price']] = data[['ltp', 'spot_price', 'future_price']].astype(float)

    # Seprate call data and put data
    ce_data = data[data['option_type'] == 'CE']
    pe_data = data[data['option_type'] == 'PE']

    # use funcation for calculate iv seprately spot and future price
    ce_data['implied_volatility_spot'] = ce_data.apply(implied_volatility_spot_call, axis=1)
    pe_data['implied_volatility_spot'] = pe_data.apply(implied_volatility_spot_put, axis=1)

    ce_data['implied_volatility_future'] = ce_data.apply(implied_volatility_future_call, axis=1)
    pe_data['implied_volatility_future'] = pe_data.apply(implied_volatility_future_put, axis=1)

    # merge call and put data in one dataframe 
    data = pd.concat([ce_data, pe_data], ignore_index=True).sort_values(by='time_entry')
    data['implied_volatility_future'] = data['implied_volatility_future']*100
    data['implied_volatility_spot'] = data['implied_volatility_spot']*100
    
    return data

# Save data in mongodb database

In [13]:
def mongodb_save(data,today_date_str,data_base_name , collection_name):
    # Create database name
    database_name = f'{data_base_name}_option_data_{today_date_str}'
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[database_name]
    coll = db[collection_name]
    dt = data.to_dict(orient='records')
    coll.insert_many(dt)

In [14]:
def main():
    today_date = datetime.today().date()
    today_date_str = today_date.strftime("%d%m%Y")
    #today_date_str = '31052024'
    option_path = f'Y:\\{today_date_str}\\option_data.txt'

    # Find expiry wise token file path all expiry (bank-nifty , nifty , fin-nifty ,mid-nify)
    exp_path_list = find_expiry_path(today_date_str = today_date_str)

    # Find expiry wise token dataframe expiry (bank-nifty , nifty , fin-nifty , mid-nifty)
    token_dics = find_token_data(exp_path_list =exp_path_list)

    # Find Future tokens (bank-nifty , nifty , fin-nifty, mid-nifty)
    bank_fut_token = fut_price_token(today_date_str = today_date_str , index_name = 'BANKNIFTY')
    nifty_fut_token = fut_price_token(today_date_str = today_date_str , index_name = 'NIFTY')
    fin_nifty_fut_token = fut_price_token(today_date_str = today_date_str , index_name = 'FINNIFTY')
    mid_nifty_fut_token = fut_price_token(today_date_str = today_date_str , index_name = 'MIDCPNIFTY')

    # This is a funcation to find options data 
    data_dict = process_data(file_path = option_path , token_dics =token_dics , bank_fut_token =bank_fut_token , nifty_fut_token = nifty_fut_token
                 , fin_nifty_fut_token = fin_nifty_fut_token , mid_nifty_fut_token =mid_nifty_fut_token , chunk_size=100000)

    # This is a bank-nifty option data (weekly , next-weekly , monthly and future data)
    weekly_bank_nifty_data = pd.concat(data_dict['weekly_bank_nifty_data']).reset_index(drop = 'First')
    next_weekly_bank_nifty_data = pd.concat(data_dict['next_weekly_bank_nifty_data']).reset_index(drop = 'First')
    monthly_bank_nifty_data = pd.concat(data_dict['monthly_bank_nifty_data']).reset_index(drop = 'First')
    bank_fut_price = pd.concat(data_dict['bank_fut_price']).reset_index(drop = 'First')

     # This is a nifty option data (weekly , next-weekly , monthly and future data)
    weekly_nifty_data = pd.concat(data_dict['weekly_nifty_data']).reset_index(drop = 'First')
    next_weekly_nifty_data = pd.concat(data_dict['next_weekly_nifty_data']).reset_index(drop = 'First')
    monthly_nifty_data = pd.concat(data_dict['monthly_nifty_data']).reset_index(drop = 'First')
    nifty_fut_price = pd.concat(data_dict['nifty_fut_price']).reset_index(drop = 'First')
    
    # This is a fin-nifty option data (weekly , next-weekly , monthly and future data)
    weekly_fin_nifty_data = pd.concat(data_dict['weekly_fin_nifty_data']).reset_index(drop = 'First')
    next_weekly_fin_nifty_data = pd.concat(data_dict['next_weekly_fin_nifty_data']).reset_index(drop = 'First')
    monthly_fin_nifty_data = pd.concat(data_dict['monthly_fin_nifty_data']).reset_index(drop = 'First')
    fin_nifty_fut_price = pd.concat(data_dict['fin_nifty_fut_price']).reset_index(drop = 'First')

    # This is a mid-nifty option data (weekly , next-weekly , monthly and future data)
    weekly_mid_nifty_data = pd.concat(data_dict['weekly_mid_nifty_data']).reset_index(drop = 'First')
    next_weekly_mid_nifty_data = pd.concat(data_dict['next_weekly_mid_nifty_data']).reset_index(drop = 'First')
    monthly_mid_nifty_data = pd.concat(data_dict['monthly_mid_nifty_data']).reset_index(drop = 'First')
    mid_nifty_fut_price = pd.concat(data_dict['mid_nifty_fut_price']).reset_index(drop = 'First')

    # Find bank-nifty token details weekly and next weekly and monthly
    bank_nifty_token_dic_weekly = token_dics['weekly']['BANKNIFTY']
    bank_nifty_token_dic_next_weekly = token_dics['next_weekly']['BANKNIFTY']
    bank_nifty_token_dic_monthly = token_dics['monthly']['BANKNIFTY']

    # Find nifty token details weekly and next weekly and monthly
    nifty_token_dic_weekly = token_dics['weekly']['NIFTY']
    nifty_token_dic_next_weekly = token_dics['next_weekly']['NIFTY']
    nifty_token_dic_monthly = token_dics['monthly']['NIFTY']

    # Find fin-nifty token details weekly and next weekly and monthly
    fin_nifty_token_dic_weekly = token_dics['weekly']['FINNIFTY']
    fin_nifty_token_dic_next_weekly = token_dics['next_weekly']['FINNIFTY']
    fin_nifty_token_dic_monthly = token_dics['monthly']['FINNIFTY']

    # Find mid-nifty token details weekly and next weekly and monthly
    mid_nifty_token_dic_weekly = token_dics['weekly']['MIDNIFTY']
    mid_nifty_token_dic_next_weekly = token_dics['next_weekly']['MIDNIFTY']
    mid_nifty_token_dic_monthly = token_dics['monthly']['MIDNIFTY']
    
    start_time = time.time()
    
    # Merge Token details bank nifty weekly and next weekly and monthly (strike price , option type expiry and Future price)
    bank_data_weekly = token_deatils_merge(data = weekly_bank_nifty_data ,token_data = bank_nifty_token_dic_weekly ,fut_data =  bank_fut_price )
    bank_data_next_weekly = token_deatils_merge(data = next_weekly_bank_nifty_data ,token_data = bank_nifty_token_dic_next_weekly ,fut_data =  bank_fut_price )
    bank_data_monthly = token_deatils_merge(data = monthly_bank_nifty_data , token_data = bank_nifty_token_dic_monthly , fut_data =  bank_fut_price )

    # Merge Token deatils nifty weekly and next weekly and monthly (strike price , option type expiry and Future price)
    nifty_data_weekly = token_deatils_merge(data = weekly_nifty_data ,token_data = nifty_token_dic_weekly ,fut_data =  nifty_fut_price )
    nifty_data_next_weekly = token_deatils_merge(data = next_weekly_nifty_data ,token_data = nifty_token_dic_next_weekly ,fut_data =  nifty_fut_price )
    nifty_data_monthly = token_deatils_merge(data = monthly_nifty_data ,token_data = nifty_token_dic_monthly ,fut_data =  nifty_fut_price )

    # Merge Token deatils Fin-nifty weekly and next weekly and monthly (strike price , option type expiry and Future price)
    fin_nifty_weekly = token_deatils_merge(data = weekly_fin_nifty_data ,token_data = fin_nifty_token_dic_weekly ,fut_data =  fin_nifty_fut_price )
    fin_nifty_next_weekly = token_deatils_merge(data = next_weekly_fin_nifty_data ,token_data = fin_nifty_token_dic_next_weekly ,fut_data =  fin_nifty_fut_price )
    fin_nifty_monthly = token_deatils_merge(data = monthly_fin_nifty_data ,token_data = fin_nifty_token_dic_monthly ,fut_data =  fin_nifty_fut_price )

    # Merge Token deatils mid-nifty weekly and next weekly and monthly (strike price , option type expiry and Future price)
    mid_nifty_weekly = token_deatils_merge(data = weekly_mid_nifty_data ,token_data = mid_nifty_token_dic_weekly ,fut_data =  mid_nifty_fut_price )
    mid_nifty_next_weekly = token_deatils_merge(data = next_weekly_mid_nifty_data ,token_data = mid_nifty_token_dic_next_weekly ,fut_data =  mid_nifty_fut_price )
    mid_nifty_monthly = token_deatils_merge(data = monthly_mid_nifty_data ,token_data = mid_nifty_token_dic_monthly ,fut_data =  mid_nifty_fut_price )

    # add spot price in datas

    # Bank nifty weekly and next weekly and monthly
    main_data_bank_weekly  = spot_price_merge(data =bank_data_weekly , index_name = 'NiftyBank' , today_date_str = today_date_str)
    main_data_bank_next_weekly  = spot_price_merge(data =bank_data_next_weekly , index_name = 'NiftyBank' , today_date_str = today_date_str)
    main_data_bank_monthly = spot_price_merge(data =bank_data_monthly , index_name = 'NiftyBank' , today_date_str = today_date_str)

    # Nifty weekly and next weekly and monthly
    main_data_nifty_weekly = spot_price_merge(data =nifty_data_weekly , index_name = 'Nifty50' , today_date_str = today_date_str)
    main_data_nifty_next_weekly = spot_price_merge(data =nifty_data_next_weekly , index_name = 'Nifty50' , today_date_str = today_date_str)
    main_data_nifty_monthly = spot_price_merge(data =nifty_data_monthly , index_name = 'Nifty50' , today_date_str = today_date_str)

    # Fin-nifty weekly and next weekly and monthly
    main_data_fin_nifty_weekly = spot_price_merge(data =fin_nifty_weekly , index_name = 'NiftyFinService' , today_date_str = today_date_str)
    main_data_fin_nifty_next_weekly = spot_price_merge(data =fin_nifty_next_weekly , index_name = 'NiftyFinService' , today_date_str = today_date_str)
    main_data_fin_nifty_monthly = spot_price_merge(data =fin_nifty_monthly , index_name = 'NiftyFinService' , today_date_str = today_date_str)

    # Mid-nifty weekly and next weekly and monthly 
    main_data_mid_nifty_weekly = spot_price_merge(data =mid_nifty_weekly , index_name = 'NIFTYMIDSELECT' , today_date_str = today_date_str)
    main_data_mid_nifty_next_weekly = spot_price_merge(data =mid_nifty_next_weekly , index_name = 'NIFTYMIDSELECT' , today_date_str = today_date_str)
    main_data_mid_nifty_monthly = spot_price_merge(data =mid_nifty_monthly , index_name = 'NIFTYMIDSELECT' , today_date_str = today_date_str)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time:.2f} seconds")
    
    # Find Bank-Nifty IV 
    main_data_bank_weekly = calculate_iv(data = main_data_bank_weekly)
    main_data_bank_next_weekly = calculate_iv(data = main_data_bank_next_weekly)
    main_data_bank_monthly = calculate_iv(data = main_data_bank_monthly)

    # Find Nifty IV
    main_data_nifty_weekly = calculate_iv(data = main_data_nifty_weekly)
    main_data_nifty_next_weekly = calculate_iv(data = main_data_nifty_next_weekly)
    main_data_nifty_monthly = calculate_iv(data = main_data_nifty_monthly)

    # Find Fin-Nifty IV
    main_data_fin_nifty_weekly = calculate_iv(data = main_data_fin_nifty_weekly)
    main_data_fin_nifty_next_weekly = calculate_iv(data = main_data_fin_nifty_next_weekly)
    main_data_fin_nifty_monthly = calculate_iv(data = main_data_fin_nifty_monthly)

    # Find Mid-Nifty IV
    main_data_mid_nifty_weekly = calculate_iv(data = main_data_mid_nifty_weekly)
    main_data_mid_nifty_next_weekly = calculate_iv(data = main_data_mid_nifty_next_weekly)
    main_data_mid_nifty_monthly = calculate_iv(data = main_data_mid_nifty_monthly)
    
    # Bank nifty data save in mongodb (weekly , next-weekly,monthly)
    mongodb_save(data = main_data_bank_weekly , today_date_str = today_date_str , data_base_name = 'banknifty' , collection_name = 'weekly')
    mongodb_save(data = main_data_bank_next_weekly , today_date_str = today_date_str , data_base_name = 'banknifty' , collection_name = 'next_weekly')
    mongodb_save(data = main_data_bank_monthly , today_date_str = today_date_str , data_base_name = 'banknifty' , collection_name = 'monthly')

    # Nifty data save in mongodb (weekly , next-weekly,monthly)
    mongodb_save(data = main_data_nifty_weekly , today_date_str = today_date_str , data_base_name = 'nifty' , collection_name = 'weekly')
    mongodb_save(data = main_data_nifty_next_weekly , today_date_str = today_date_str , data_base_name = 'nifty' , collection_name = 'next_weekly')
    mongodb_save(data = main_data_nifty_monthly , today_date_str = today_date_str , data_base_name = 'nifty' , collection_name = 'monthly')

    # Fin-Nifty data save in mongodb (weekly , next-weekly,monthly)
    mongodb_save(data = main_data_fin_nifty_weekly , today_date_str = today_date_str , data_base_name = 'fin-nifty' , collection_name = 'weekly')
    mongodb_save(data = main_data_fin_nifty_next_weekly , today_date_str = today_date_str , data_base_name = 'fin-nifty' , collection_name = 'next_weekly')
    mongodb_save(data = main_data_fin_nifty_monthly, today_date_str = today_date_str , data_base_name = 'fin-nifty' , collection_name = 'monthly')

    # Mid-Nifty data save in mongodb (weekly , next-weekly,monthly)
    mongodb_save(data = main_data_mid_nifty_weekly , today_date_str = today_date_str , data_base_name = 'mid-nifty' , collection_name = 'weekly')
    mongodb_save(data = main_data_mid_nifty_next_weekly , today_date_str = today_date_str , data_base_name = 'mid-nifty' , collection_name = 'next_weekly')
    mongodb_save(data = main_data_mid_nifty_monthly , today_date_str = today_date_str , data_base_name = 'mid-nifty' , collection_name = 'monthly')




In [15]:
start_time = time.time()
if __name__ == '__main__':
    main()
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

{'BANKNIFTY': 'Y:\\daily_contract_file\\03062024\\BANKNIFTY_05062024.csv', 'NIFTY': 'Y:\\daily_contract_file\\03062024\\NIFTY_06062024.csv', 'FINNIFTY': 'Y:\\daily_contract_file\\03062024\\FINNIFTY_04062024.csv', 'MIDCPNIFTY': 'Y:\\daily_contract_file\\03062024\\MIDCPNIFTY_03062024.csv'}
weekly
{'BANKNIFTY': 'Y:\\daily_contract_file\\03062024\\BANKNIFTY_12062024.csv', 'NIFTY': 'Y:\\daily_contract_file\\03062024\\NIFTY_13062024.csv', 'FINNIFTY': 'Y:\\daily_contract_file\\03062024\\FINNIFTY_11062024.csv', 'MIDCPNIFTY': 'Y:\\daily_contract_file\\03062024\\MIDCPNIFTY_10062024.csv'}
next_weekly
{'BANKNIFTY': 'Y:\\daily_contract_file\\03062024\\BANKNIFTY_26062024.csv', 'NIFTY': 'Y:\\daily_contract_file\\03062024\\NIFTY_29062028.csv', 'FINNIFTY': 'Y:\\daily_contract_file\\03062024\\FINNIFTY_25062024.csv', 'MIDCPNIFTY': 'Y:\\daily_contract_file\\03062024\\MIDCPNIFTY_24062024.csv'}
monthly
Elapsed time: 98.33 seconds
Elapsed time: 10618.11 seconds


In [34]:
today_date_str = '03062024'
month = datetime.today().month
year = datetime.today().year
data_path = f'Y:\\daily_contract_file\\{today_date_str}\\*'

# Read file paths into a DataFrame
file_paths = pd.DataFrame(glob(data_path), columns=['path'])

# Extract split_data, index_name, and expiry using a single apply
file_paths['split_data'] = file_paths['path'].apply(lambda x: x.split('\\')[-1])
file_paths[['index_name', 'expiry']] = file_paths['split_data'].str.extract(r'([^_]+)_(\d{8})')

# Remove rows where expiry is 'index'
file_paths = file_paths[file_paths['expiry'] != 'index']

# Convert 'expiry' to datetime and sort the DataFrame
file_paths['expiry'] = pd.to_datetime(file_paths['expiry'], format='%d%m%Y')
file_paths = file_paths.sort_values(by='expiry')

# Create separate DataFrames for each index_name
index_names = ['BANKNIFTY', 'NIFTY', 'FINNIFTY', 'MIDCPNIFTY']
separated_data = {index: file_paths[file_paths['index_name'] == index].reset_index(drop=True)
                  for index in index_names}

# Accessing the DataFrames
bank_nifty = separated_data['BANKNIFTY']
nifty = separated_data['NIFTY']
fin_nifty = separated_data['FINNIFTY']
mid_nifty = separated_data['MIDCPNIFTY']
index_list = [bank_nifty ,nifty , fin_nifty , mid_nifty ]
weekly_list = {}
next_weekly_list = {}
monthly_list = {}

for i in index_list :
    try:
        
        name = i['index_name'].iloc[0]
        weekly  = i['path'][0]
        next_weekly = i['path'][1]
        monthly = i[(i['expiry'].dt.month == month) & (i['expiry'].dt.year == year)].reset_index(drop = 'First')['path'].iloc[-1]
        
        if monthly is None:
            monthly = i[i['expiry'].dt.month == (month + 1)]['path'].iloc[-1]
    except Exception as e:
        monthly = i[(i['expiry'].dt.month == (month + 1)) & (i['expiry'].dt.year == year)].reset_index(drop = 'First')['path'].iloc[-1]
        
    weekly_list.update({ name : weekly})
    next_weekly_list.update({ name : next_weekly})
    monthly_list.update({ name : monthly})

In [35]:
monthly_list

{'BANKNIFTY': 'Y:\\daily_contract_file\\03062024\\BANKNIFTY_26062024.csv',
 'NIFTY': 'Y:\\daily_contract_file\\03062024\\NIFTY_27062024.csv',
 'FINNIFTY': 'Y:\\daily_contract_file\\03062024\\FINNIFTY_25062024.csv',
 'MIDCPNIFTY': 'Y:\\daily_contract_file\\03062024\\MIDCPNIFTY_24062024.csv'}

In [20]:
file_paths.sort_values(by='expiry').reset_index(drop ='First')

,path,split_data,index_name,expiry
0,Y:\daily_contract_file\03062024\MIDCPNIFTY_030...,MIDCPNIFTY_03062024.csv,MIDCPNIFTY,2024-06-03
1,Y:\daily_contract_file\03062024\contract_03062...,contract_03062024.txt,contract,2024-06-03
2,Y:\daily_contract_file\03062024\FINNIFTY_04062...,FINNIFTY_04062024.csv,FINNIFTY,2024-06-04
3,Y:\daily_contract_file\03062024\BANKNIFTY_0506...,BANKNIFTY_05062024.csv,BANKNIFTY,2024-06-05
4,Y:\daily_contract_file\03062024\NIFTY_06062024...,NIFTY_06062024.csv,NIFTY,2024-06-06
5,Y:\daily_contract_file\03062024\MIDCPNIFTY_100...,MIDCPNIFTY_10062024.csv,MIDCPNIFTY,2024-06-10
6,Y:\daily_contract_file\03062024\FINNIFTY_11062...,FINNIFTY_11062024.csv,FINNIFTY,2024-06-11
7,Y:\daily_contract_file\03062024\BANKNIFTY_1206...,BANKNIFTY_12062024.csv,BANKNIFTY,2024-06-12
8,Y:\daily_contract_file\03062024\NIFTY_13062024...,NIFTY_13062024.csv,NIFTY,2024-06-13
9,Y:\daily_contract_file\03062024\MIDCPNIFTY_140...,MIDCPNIFTY_14062024.csv,MIDCPNIFTY,2024-06-14


In [24]:
data = file_paths[file_paths['index_name'] == 'NIFTY'].reset_index(drop ='First')

In [31]:
data[(data['expiry'].dt.month == month) & (data['expiry'].dt.year ==year)]

,path,split_data,index_name,expiry
0,Y:\daily_contract_file\03062024\NIFTY_06062024...,NIFTY_06062024.csv,NIFTY,2024-06-06
1,Y:\daily_contract_file\03062024\NIFTY_13062024...,NIFTY_13062024.csv,NIFTY,2024-06-13
2,Y:\daily_contract_file\03062024\NIFTY_20062024...,NIFTY_20062024.csv,NIFTY,2024-06-20
3,Y:\daily_contract_file\03062024\NIFTY_27062024...,NIFTY_27062024.csv,NIFTY,2024-06-27
